In [1]:
import pandas as pd
data = pd.read_csv('car_evaluation.csv')
data.head()

,buying,maint,doors,persons,lug_boot,safety,outcome
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [2]:
X = data.iloc[:, :-1]
Y = data.outcome
X.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [3]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.loc[:,['buying','maint', 'doors', 'persons', 'lug_boot','safety']] = \
X.loc[:,['buying','maint', 'doors', 'persons', 'lug_boot','safety']].apply(enc.fit_transform)
X.head()

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,2,1
1,3,3,0,0,2,2
2,3,3,0,0,2,0
3,3,3,0,0,1,1
4,3,3,0,0,1,2


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

In [5]:
data.outcome.value_counts()

outcome
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

In [6]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

不使用SMOTE进行模型训练

In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_predict))
pd.crosstab(y_test,y_predict)

0.9267822736030829


col_0,acc,good,unacc,vgood
outcome,,,,
acc,82,1,19,0
good,6,14,1,0
unacc,2,0,369,0
vgood,3,1,5,16


使用SMOTE进行模型训练

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [9]:
X_train_smote, y_train_smote = smote.fit_resample(X_train.astype(float), y_train)

In [10]:
from collections import Counter
print("不使用SMOTE的样本数目：",Counter(y_train))
print("使用SMOTE的样本数目：",Counter(y_train_smote))

不使用SMOTE的样本数目： Counter({'unacc': 839, 'acc': 282, 'good': 48, 'vgood': 40})
使用SMOTE的样本数目： Counter({'acc': 839, 'unacc': 839, 'vgood': 839, 'good': 839})


In [11]:
model = KNeighborsClassifier()
model.fit(X_train_smote, y_train_smote)
y_predict = model.predict(X_test)

In [12]:
print(accuracy_score(y_test, y_predict))
pd.crosstab(y_test,y_predict)

0.9344894026974951


col_0,acc,good,unacc,vgood
outcome,,,,
acc,86,6,8,2
good,2,19,0,0
unacc,13,1,356,1
vgood,1,0,0,24


准确率提升了很多，说明SMOTE方法可以有效地处理过拟合问题。